# Game of Sets - largest card deck with no matches

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# initialize card deck df
card_names = pd.DataFrame(columns=['color', 'shape', 'fill', 'number'])
card_names['color'] = ['red', 'green', 'purple']
card_names['shape'] = ['oval', 'diamond', 'squiggle']
card_names['fill'] = ['solid', 'striped', 'empty']
card_names['number'] = [1, 2, 3]


In [5]:
card_names.head(3)

,color,shape,fill,number
0,red,oval,solid,1
1,green,diamond,striped,2
2,purple,squiggle,empty,3


In [67]:
# prepopulate short card deck df with all cards using numbers 0, 1, 2 for each attribute
card_deck = pd.DataFrame(columns=['color', 'shape', 'fill', 'number'])
card_deck['color'] = [0, 1, 2] * 3
card_deck['shape'] = [1, 2, 0, 1, 1, 2, 0, 2, 0]
card_deck['fill'] = [2, 0, 1, 2, 0, 1, 2, 0, 1]
card_deck['number'] = [0, 1, 2, 2, 0, 1, 2, 0, 1]


In [84]:
# prepopulate complete card deck df with all possible combinations  using numbers 0, 1, 2 for each of four attributes
card_deck = pd.DataFrame(columns=['color', 'shape', 'fill', 'number'])
card_deck['color'] = [0] * 27 + [1] * 27 + [2] * 27
card_deck['shape'] = ([0] * 9 + [1] * 9 + [2] * 9 ) * 3
card_deck['fill'] =  ([0] * 3 + [1] * 3 + [2] * 3 ) * 9
card_deck['number'] = [0, 1, 2] * 27

In [ ]:
card_deck.head(46)

In [82]:
len(card_deck)

81

In [9]:
# trim_matches function takes in a card deck and returns a card deck with all matches removed
def trim_matches(card_deck):
    # initialize empty list to store indices of matches
    matches = []
    # loop through all cards in deck
    for i in range(len(card_deck)):
        # loop through all cards in deck
        for j in range(len(card_deck)):
            # check if cards are the same card
            if i == j:
                continue
            # check if cards are a match
            if card_deck.iloc[i].equals(card_deck.iloc[j]):
                # add index of match to matches list
                matches.append(i)
                matches.append(j)
    # drop matches from card deck
    card_deck = card_deck.drop(matches)
    # reset index
    card_deck = card_deck.reset_index(drop=True)
    # return card deck
    return card_deck

In [23]:
trim_matches(card_deck)

,color,shape,fill,number
0,0,1,2,0
1,1,2,0,1
2,2,0,1,2
3,0,1,2,2
4,1,1,0,0
5,2,2,1,1
6,0,0,2,2
7,1,2,0,0
8,2,0,1,1


In [25]:
card1 = card_deck.iloc[1]
card2 = card_deck.iloc[2]
card3 = card_deck.iloc[3]
print(card1, card2, card3)

color     1
shape     2
fill      0
number    1
Name: 1, dtype: int64 color     2
shape     0
fill      1
number    2
Name: 2, dtype: int64 color     0
shape     1
fill      2
number    2
Name: 3, dtype: int64


In [66]:
# is_set function takes in three cards and returns True if they are a set, False otherwise
def is_set(card1, card2, card3):
    # check if all cards are the same card
    if card1.equals(card2) and card2.equals(card3):
        return False
    # check if any two cards are the same card
    if card1.equals(card2) or card2.equals(card3) or card1.equals(card3):
        return False
    # check if any two cards are not a set
    if (card1['color'] + card2['color'] + card3['color']) % 3 != 0:
        return False
    if (card1['shape'] + card2['shape'] + card3['shape']) % 3 != 0:
        return False
    if (card1['fill'] + card2['fill'] + card3['fill']) % 3 != 0:
        return False
    if (card1['number'] + card2['number'] + card3['number']) % 3 != 0:
        return False
    # if all checks pass, return True
    return True

In [ ]:
a = 6
b = 0
c = 4

print(card_deck.iloc[[a, b, c]])
is_set(card_deck.iloc[a], card_deck.iloc[b], card_deck.iloc[c])

In [42]:
# trim_sets function takes in a card deck and two cards selected by their index and returns a card deck with all cards making a set with those two cards removed
def trim_sets(card_deck, card1_index, card2_index):
    # check that card1_index and card2_index are in range
    if card1_index >= len(card_deck) or card2_index >= len(card_deck):
        print('Error: card index out of range')
        return card_deck
    # initialize empty list to store indices of sets
    sets = []
    # loop through all cards in deck
    for i in range(len(card_deck)):
        # check if cards are a set
        if is_set(card_deck.iloc[card1_index], card_deck.iloc[card2_index], card_deck.iloc[i]):
            # add index of set to sets list
            sets.append(i)
    # drop sets from card deck
    card_deck = card_deck.drop(sets)
    # reset index
    card_deck = card_deck.reset_index(drop=True)
    # return card deck
    return card_deck

In [ ]:
card1_index = 0
card2_index = 3
if card1_index >= len(card_deck) or card2_index >= len(card_deck):
        print('Error: card index out of range')
print(card_deck.iloc[[card1_index, card2_index]])

print(card_deck)
card_deck = trim_sets(card_deck, card1_index, card2_index)
print(card_deck)

In [85]:
print('starting deck size:', len(card_deck))
# loop through card deck using all possible combinations of two cards
for i in range(len(card_deck)):
    # exit loop if i is greater than the current card deck length
    if i >= len(card_deck):
        break
    for j in range(len(card_deck)):
        # exit loop if j is greater than the current card deck length
        if j >= len(card_deck):
            break
        # trim card deck of all sets with those two cards
        card_deck = trim_sets(card_deck, i, j)

starting deck size: 81


In [86]:
card_deck

,color,shape,fill,number
0,0,0,0,0
1,0,0,0,1
2,0,0,1,0
3,0,0,1,1
4,0,1,0,0
5,0,1,0,1
6,0,1,1,0
7,0,1,1,1
8,1,0,0,0
9,1,0,0,1
